In [1]:
import pandas as pd
from pathlib import Path, PurePath
from datetime import datetime, date
import re
from collections import defaultdict

import numpy as np
import xlrd
import openpyxl

DOCS_FOLDER = Path(r'D:\4. Programming\ProjectsSoftware\2_ai\get_reports\docs')
test_folder_path = Path(DOCS_FOLDER / 'ABEO') 


In [2]:
Tiker = 'ABEO'

In [3]:
from pathlib import Path
import win32com.client as win32
    
def convert_xls_to_xlsx(path: Path) -> None:
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(path.absolute())

    # FileFormat=51 is for .xlsx extension
    wb.SaveAs(str(path.absolute().with_suffix(".xlsx")), FileFormat=51)
    wb.Close()
    excel.Application.Quit()

In [4]:

coverted_files = [convert_xls_to_xlsx(_file) for _file in test_folder_path.iterdir()]



In [5]:
_files = [_file for _file in test_folder_path.iterdir() if _file.suffix == '.xlsx']


In [6]:
_files[3]

WindowsPath('D:/4. Programming/ProjectsSoftware/2_ai/get_reports/docs/ABEO/Financial_Report (4).xlsx')

In [12]:
_path = r'D:\4. Programming\ProjectsSoftware\2_ai\get_reports\docs\ABEO\Financial_Report (2).xlsx'

In [8]:
pd.read_excel(_files[3])

,Document and Entity Information,9 Months Ended,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,"Sep. 30, 2012","Nov. 14, 2012","Nov. 14, 2012","Nov. 14, 2012"
1,NaN,NaN,Series A Convertible Preferred Stock [Member],Series B Convertible Preferred Stock [Member],Common Stock [Member]
2,Entity Registrant Name,ACCESS PHARMACEUTICALS INC,NaN,NaN,NaN
3,Entity Central Index Key,318306,NaN,NaN,NaN
4,Current Fiscal Year End Date,-19,NaN,NaN,NaN
5,Entity Well-known Seasoned Issuer,No,NaN,NaN,NaN
6,Entity Voluntary Filers,No,NaN,NaN,NaN
7,Entity Current Reporting Status,Yes,NaN,NaN,NaN
8,Entity Filer Category,Smaller Reporting Company,NaN,NaN,NaN
9,"Entity Common Stock, Shares Outstanding",NaN,"2,938.3617",1,"24,196,734"


In [10]:
def check_threee_moth_ended(data_frame):
    
    count_ended = {}
    check_columns = []
    have_found =0 
    for indexx,i in enumerate(data_frame.columns):
        second_row = data_frame[i][0]    
        if re.search('Months Ended',i):
            count_ended[indexx] = i
            have_found +=1
    for indexx,i in enumerate(data_frame.columns):
        second_row = data_frame[i][0]    
        if have_found>0:
            data_frame.set_index
        
            try:
                
                new_columns = data_frame.columns[indexx] + ' ' +  second_row
 
            except:
                new_columns = data_frame.columns[indexx]

            check_columns.append(new_columns)

        else:
            
            check_columns.append(i)
            
    data_frame.columns  = check_columns
    if have_found >0:
        data_frame = data_frame.iloc[1:]

    else:
        pass

    
    return data_frame#.iloc[:,1:]

In [25]:
excl_add = 'Condensed'
exclude_report_list_before_join = ['Net Income per Share',
                       'Summary of Significant Accounting Policies','Common Stock'] 
exclude_report_list = '|'.join(exclude_report_list_before_join)
excludeded_keys = []

tables_pure_numeric = defaultdict(list)   
tables_other        = defaultdict(list)  

REALY_NAMES =  {}

files = _files
for period in files:
    
    wb = openpyxl.load_workbook(period)
    sheetnames_list = wb.sheetnames

    for sheetname in sheetnames_list:
        sheet = wb[sheetname]

        if len(str(sheet['b4'].value))<9 and len(str(sheet['b3'].value))<9:

            header = sheet['A1'].value
        
 

            if re.search(exclude_report_list,header):
                excludeded_keys.append(header)
                pass
            else :
                if re.search(excl_add,header):
                    
                    spisok = header.split(' ')
                    spisok.remove(excl_add)
                # (spisok.index(keysss))
                    header=' '.join(spisok)
                else:
                    pass

                REALY_NAMES[header] = sheetname
                a = check_threee_moth_ended(pd.read_excel(period,sheet_name=sheetname))
                
                tables_pure_numeric.setdefault(header, []).append(a)
        else:
            header = sheet['A1'].value
            tables_other
            REALY_NAMES[header] = sheetname
            a = check_threee_moth_ended(pd.read_excel(period,sheet_name=sheetname))
                
            tables_other.setdefault(header, []).append(a)


print('OUTPUT',len(tables_pure_numeric),'|||', 'INPUT',len(wb.sheetnames))   
for b,i in enumerate(REALY_NAMES.keys()):
    print(b+1,i)

OUTPUT 15 ||| INPUT 13
1 Document And Entity Information (USD  $)
2 Condensed Consolidated Balance Sheets (Unaudited) (USD  $)
3 Condensed Consolidated Balance Sheets (Unaudited) (Parenthetical) (USD  $)
4 Condensed Consolidated Statements of Operations (Unaudited) (USD  $)
5 Consolidated Statements of Stockholders' Deficit (Unaudited) (USD  $)
6 Condensed Consolidated Statements of Cash Flows (Unaudited) (USD  $)
7 Interim Financial Statements
8 Intangible Assets
9 Notes Payable
10 Liquidity
11 Fair Value of Financial Instruments
12 Stock Based Compensation
13 Subsequent Events (Unaudited)
14 Document And Entity Information (USD $)
15 CONSOLIDATED BALANCE SHEETS (USD $)
16 CONSOLIDATED BALANCE SHEETS (Parenthetical) (USD $)
17 CONSOLIDATED STATEMENTS OF OPERATIONS (USD $)
18 CONSOLIDATED STATEMENTS OF STOCKHOLDERS' DEFICIT (USD $)
19 CONSOLIDATED STATEMENTS OF CASH FLOWS (USD $)
20 NATURE OF OPERATIONS AND SUMMARY OF SIGNIFICANT ACCOUNTING POLICIES
21 LIQUIDITY
22 RELATED PARTY TRANSA

In [26]:
[f'{frame_name} {len(tables_pure_numeric[frame_name])}' for frame_name in REALY_NAMES.keys() if len(tables_pure_numeric[frame_name])>1]

["Consolidated Statements of Stockholders' Deficit (Unaudited) (USD \xa0$) 2",
 'Interim Financial Statements 4',
 'Consolidated Balance Sheets (unaudited) (USD $) 2',
 'Consolidated Statements of Operations (unaudited) (USD $) 2',
 'Intangible Assets (Details) (USD $) 2',
 'Note Payable (Details) (USD $) 2',
 'Fair Value of Financial Instruments (Details) (USD $) 2',
 'Stock Based Compensation (Details) (USD $) 2']

In [20]:
# finder = Report name
finder = "Interim Financial Statements"
print(f'App have found {len(tables_pure_numeric[finder])} documents from\n{REALY_NAMES[finder]}')

App have found 4 documents from
Interim_Financial_Statements


In [27]:
REALY_NAMES

{'Document And Entity Information (USD \xa0$)': 'Document_And_Entity_Informatio',
 'Condensed Consolidated Balance Sheets (Unaudited) (USD \xa0$)': 'Condensed_Consolidated_Balance',
 'Condensed Consolidated Balance Sheets (Unaudited) (Parenthetical) (USD \xa0$)': 'Condensed_Consolidated_Balance1',
 'Condensed Consolidated Statements of Operations (Unaudited) (USD \xa0$)': 'Condensed_Consolidated_Stateme',
 "Consolidated Statements of Stockholders' Deficit (Unaudited) (USD \xa0$)": 'Condensed_Consolidated_Stateme1',
 'Condensed Consolidated Statements of Cash Flows (Unaudited) (USD \xa0$)': 'Condensed_Consolidated_Stateme2',
 'Interim Financial Statements': 'Interim_Financial_Statements',
 'Intangible Assets': 'Intangible_Assets',
 'Notes Payable': 'Notes_Payable',
 'Liquidity': 'Liquidity',
 'Fair Value of Financial Instruments': 'Fair_Value_of_Financial_Instru',
 'Stock Based Compensation': 'Stock_Based_Compensation',
 'Subsequent Events (Unaudited)': 'Subsequent_Events_Unaudited',
 '

## Intangible Assets

In [46]:

sorted(
    {
        frame_name : len(tables_other[frame_name]) 
        for frame_name 
        in REALY_NAMES.keys() 
        if len(tables_other[frame_name])>1
    }
    .items(),
    key=lambda x:x[1],reverse=True)


[('Intangible Assets', 4),
 ('Liquidity', 4),
 ('Fair Value of Financial Instruments', 4),
 ('Stock Based Compensation', 4),
 ('Document and Entity Information', 3),
 ('Condensed Consolidated Balance Sheets (unaudited) (Parenthetical) (USD $)',
  3),
 ("Condensed Consolidated Statements of Stockholders' Deficit (unaudited) (USD $)",
  3),
 ('Condensed Consolidated Statements of Cash Flows (unaudited) (USD $)', 3),
 ('Document And Entity Information (USD \xa0$)', 2),
 ('Condensed Consolidated Balance Sheets (Unaudited) (USD \xa0$)', 2),
 ('Condensed Consolidated Balance Sheets (Unaudited) (Parenthetical) (USD \xa0$)',
  2),
 ('Condensed Consolidated Statements of Operations (Unaudited) (USD \xa0$)',
  2),
 ('Condensed Consolidated Statements of Cash Flows (Unaudited) (USD \xa0$)',
  2),
 ('Subsequent Events (Unaudited)', 2),
 ('Note Payable', 2),
 ('Intangible Assets (Tables)', 2),
 ('Fair Value of Financial Instruments (Tables)', 2),
 ('Stock Based Compensation (Tables)', 2),
 ('Liquid

In [52]:
table_1 = tables_other['Intangible Assets'][0]
# table_1

In [68]:
table_1.dropna(axis=1, how='all').drop(table_1.columns[0],axis=1)

,"6 Months Ended Jun. 30, 2012",Unnamed: 3,Unnamed: 4,Unnamed: 7,Unnamed: 8,Unnamed: 11,Unnamed: 12,Unnamed: 15,Unnamed: 16
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(2) Intangible Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Intangible assets consist of the following (in...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,"June 30, 2012",NaN,NaN,NaN,"December 31, 2011",NaN,NaN,NaN
7,NaN,Gross,NaN,Accumulated,NaN,Gross,NaN,Accumulated,NaN
8,NaN,carrying,NaN,amortization,NaN,carrying,NaN,Amortization,NaN
9,NaN,value,NaN,NaN,NaN,value,NaN,NaN,NaN
10,Amortizable intangible assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Numeric_tables

In [77]:
table_1.columns

Index(['Condensed Consolidated Statements of Operations (unaudited) (USD $)',
       '3 Months Ended Sep. 30, 2012', 'Unnamed: 2 Sep. 30, 2011',
       '9 Months Ended Sep. 30, 2012', 'Unnamed: 4 Sep. 30, 2011'],
      dtype='object')

In [103]:
series_1 = table_1['3 Months Ended Sep. 30, 2012']
series_1

1                NaN
2                 62
3                781
4                 28
5                  0
6                871
7                NaN
8                308
9                 79
10         1,422,000
11                18
12         1,827,000
13              -956
14               111
15              -186
16                 0
17                64
18      (13,900,000)
19      (13,911,000)
20      (14,867,000)
21               444
22    $ (15,311,000)
23               NaN
24          $ (0.63)
25        24,173,705
Name: 3 Months Ended Sep. 30, 2012, dtype: object

In [121]:
import re
import numpy as np
ser = (
    series_1
        .str.replace(',', '')
        .str.replace('$', '')
        # .str.strip()
    
)
pattern = r"\((-?\d+(?:,\d+)*)\)"

pd.to_numeric(
    pd.Series(series_1,dtype="string")
        .str.replace(',', '')
        .str.replace('$', '')
        .apply(lambda x: re.sub(pattern, r'\1', str(x)))
    , errors='coerce')
# str.replace(',', '')
# extracted_values = ser.str.extract(pattern, expand=False)
# extracted_values.where(extracted_values.notnull(), ser)
# Apply the function to the 'Value' column
# np.apply(lambda x: re.search(pattern, x),ser)

1            NaN
2           62.0
3          781.0
4           28.0
5            0.0
6          871.0
7            NaN
8          308.0
9           79.0
10     1422000.0
11          18.0
12     1827000.0
13        -956.0
14         111.0
15        -186.0
16           0.0
17          64.0
18    13900000.0
19    13911000.0
20    14867000.0
21         444.0
22    15311000.0
23           NaN
24           NaN
25    24173705.0
Name: 3 Months Ended Sep. 30, 2012, dtype: float64